<a href="https://colab.research.google.com/github/dpant/TensorFlow/blob/main/Course_1_full_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
# GRADED FUNCTION: house_model
# 1. you can give the np.array (from numpy) to tensorflow models.
# This is coursera Course 1 Week 1 graded lab.
def house_model(y_new):
    xs = np.array([1,3,5,8,10],dtype=float)
    ys = np.array([1,2,3,4.5,5.5],dtype=float)
    model = tf.keras.Sequential([keras.layers.Dense(units=1,input_shape=[1])])
    model.compile(optimizer='sgd',loss='mean_squared_error')
    model.fit(xs,ys,epochs=10)
    model.summary()
    return model.predict(y_new)[0]

prediction = house_model([7.0])
print(prediction)


In [ ]:
# handling custom callback to stop training after certain accuracy 60% is reached.
# notice you don't have to normalize the y_train or y_test. Notice below only
# the x_train and x_test are normalized

# 2 layer (512 nodes) NN get upto 94% in training set (after 20 epochs).
# also the training accuracy increase very slowly in the later iterations.
# this dataset is harder (to train) dataset compared to digit MNIST.
# Using GPU from runtime speed up the training by 4X

print(tf.__version__)
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

mnist = tf.keras.datasets.fashion_mnist # fashon MNIST

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

callbacks = myCallback()

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, callbacks=[callbacks])

In [ ]:
# print one image
import matplotlib.pyplot as plt
def printOneExample(image):
  np.set_printoptions(linewidth=200)
  plt.imshow(image)

In [ ]:
# Multilayer (deep) NN for digit MNIST.
# Notice this two layer NN give >99% accuracy is train set and 98% in test set!

import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist # digit MINIST

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()
printOneExample(training_images[0])
training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

model.fit(training_images, training_labels, epochs=5 )

print("\n------Test accuracy-------\n")
model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])


In [ ]:
!git clone https://github.com/dpant/TensorFlow.git

In [ ]:
# Handwriting recognization 1 layer. Training accuracy 99%. test accuracy 98%
# Coursera: Tensorflow Course 1 Week 2 Lab.

import tensorflow as tf
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab mnist.npz from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location

path = f"{getcwd()}/TensorFlow/datasets/mnist.npz"
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        print("\n---\n",logs,"\n---\n")
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

# GRADED FUNCTION: train_mnist
def train_mnist():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    # YOUR CODE SHOULD START HERE
    callback = myCallback()    
    # YOUR CODE SHOULD END HERE

    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data(path=path)
    print(x_train.shape)
    # YOUR CODE SHOULD START HERE
    x_train = x_train / 255.0
    x_test = x_test/ 255.0
    # YOUR CODE SHOULD END HERE
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512,activation=tf.nn.relu),
        tf.keras.layers.Dense(10,activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # model fitting
    history = model.fit(x_train,y_train,epochs=10,callbacks=[callback]
    )
    model.evaluate(x_test,y_test)
    # model fitting
    return history.epoch, history.history['accuracy'][-1]

train_mnist()

In [ ]:
# Tensorflow Coursera Course 1 week 3
# CNN for fashion MNIST.
# CONV2-D--> MAX POOL --> CONV2-D--> MAX POOL --> DENSE(RELU) --> SOFTMAX
# Notice this gives us ~99% accuracy in train set (after 20 epoch)
# Just DNN was < 94%. This is sold 5% gain just by using convolution.
# Training time with GPU is 4s per epoch. With CPU is 100s. 25x reduction using
# GPU!!!. Conv nets are higly parallazable thus GPU training is very advantagous  
# test accuracy 90%. We are overfitting!!

import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=20)
test_loss = model.evaluate(test_images, test_labels)
